# Xephyr library is loaded and you can use all classes and methods

In [1]:
.x loadXephyr.C

(int) 0


In [2]:
errorHandler::globalPrintLevel = 1;  // print level 3 = Error only , 2 = Warning,  1= Info, 0 = Debug

In [3]:
// add path to include files to ROOT (in the file I'm going to load there are some Xephyr includes) 
gInterpreter->AddIncludePath("../src");  // in this case is just XEPHYR src from next dir.

In [ ]:
.L ../../SR1/StatisticalAnalyses/xephyr_sr1_likelihood/src/likelihoodDef.cxx

In [ ]:
pdfLikelihood likeHood = getTheLikelihood(50.);  // 50. GeV mass

In [ ]:
TCanvas *c = new TCanvas();
pdfComponent *ER = likeHood.bkg_components[0] ;
TH2F h = ER->getInterpolatedHisto();  // ER
h.Draw("colz");
c->Draw();

Now you couold for example change the value of one of the nuissance parameter and get the relative histogram interpolated... Let's do that!

In [ ]:
TCanvas *c2 = new TCanvas();
//scaleSys* S1 = (ER->myScaleUnc)[0];
LKParameter* PY = likeHood.getParameter(2);
LKParameter* RF = likeHood.getParameter(3);
PY->setCurrentValue(-0.7);               // Setting the value of RF to -1.7 sigma from its standard value (rememer the value will be interpolated)
RF->setCurrentValue(0.35);
TH2F h2 = ER->getInterpolatedHisto();
h2.Draw("colz");
c2->Draw();


##### Now let's add a Signal PdfComponent

In [ ]:
pdfComponent *Signal = likeHood.signal_component;

#### And let's draw it, just for fun...

In [ ]:
TCanvas *c3 = new TCanvas();
TH2F h3 = Signal->getInterpolatedHisto();
h3.Draw("colz");
c3->Draw();

#### Now let's add some data

In [ ]:
dataHandler *data = likeHood.data;

In [ ]:
data->drawS1S2("same*");
c3->Draw();

# Now it's time for some likelihood game

In [ ]:
//pl.maximize(false);             

likeHood.POI->setCurrentValue(2.);
likeHood.printCurrentParameters(); 
//pl.printInitialParameters();

In [ ]:
// class example to compare histograms
histoCompare p = likeHood.getModelCompare();
//p.setNameofComponent(1,"flat");
//p.setNameofComponent(1,"AC");
p.setNameofComponent(1,"ER");
p.setNameofComponent(2,"WIMP 50 GeV");
p.rebinY = 1;
p.rebinX = 3;
p.doStack = true;
p.titleY="Entries/[PE]";
p.titleX="cS2 [PE]";
p.projectionMin = 3;
p.projectionMax = 3.6;
p.projectionX = true;
//p.compareWithRatio();
TCanvas *c4 = new TCanvas();
p.compare();

c4->Draw();

In [ ]:
double qts = likeHood.maximize(false);

In [ ]:
likeHood.POI->setCurrentValue(2.);
qts = likeHood.maximize(true);